In [ ]:
#%pip install langchain langchain_experimental pandas gspread oauth2client tabulate

In [11]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.chat_models import ChatOllama
#Google sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
#

In [2]:
#Spreadsheets = name of the file
json_key_path = r"C:\Todos mis documentos\google sheets api key\key.json"
file_name = "data_homicidios"
sheet_name = "data_homicidios"


# Set up connection to Google Sheets
def connect_google_sheets(json_key_path, file_name, sheet_name):
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    
    # Key JSON
    creds = ServiceAccountCredentials.from_json_keyfile_name(json_key_path, scope)
    client = gspread.authorize(creds)

    # Spreadsheets = name of the file && worksheet = name of the sheet
    spreadsheet = client.open(file_name)
    worksheet = spreadsheet.worksheet(sheet_name)

    # data to dataframe pandas
    pd.set_option('display.max_columns', None)
    data = worksheet.get_all_records()
    df = pd.DataFrame(data)

    return df

In [3]:
df = connect_google_sheets(json_key_path, file_name, sheet_name)

In [4]:
OLLAMA_URL = "http://localhost:11434"
MODEL_NAME = "llama3.1"  # Model name

def load_llama_model():
    llm = ChatOllama(model=MODEL_NAME, base_url=OLLAMA_URL)
    return llm

llm=load_llama_model()

C:\Users\wesgb\AppData\Local\Temp\ipykernel_8436\86299712.py:5: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=MODEL_NAME, base_url=OLLAMA_URL)


In [18]:
agent = create_pandas_dataframe_agent(
    llm,                # Modelo de lenguaje
    df,                 # DataFrame
    #agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Tipo de agente
    verbose=True,        # Mostrar logs del proceso
    allow_dangerous_code=True,
    handle_parsing_errors=True
)

c:\Users\wesgb\.conda\envs\db_homicidios\lib\site-packages\langchain_experimental\agents\agent_toolkits\pandas\base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [19]:
response = agent.run("el caso numero 3 cual fue el movil?")
print(response)



> Entering new AgentExecutor chain...
Thought: Necesito encontrar la información del movimiento utilizado en el tercer caso.
Action: python_repl_ast
Action Input: print(df.loc[2, 'MOVIL'])VENGANZA
Final Answer: VENGANZA

> Finished chain.
VENGANZA


In [7]:
response = agent.run("ahora dime el distrito?")
print(response)



> Entering new AgentExecutor chain...
Thought: I need to get the value of the column "DISTRITO_HECHO" from the dataframe `df`.

Action: python_repl_ast
Action Input: print(df["DISTRITO_HECHO"])0         SAN MARTIN DE PORRES
1         SAN MARTIN DE PORRES
2      VILLA MARÍA DEL TRIUNFO
3         SAN MARTIN DE PORRES
4           LURIGANCHO-CHOSICA
                ...           
676            CERCADO DE LIMA
677                      COMAS
678              PUENTE PIEDRA
679                 CARABAYLLO
680     SAN JUAN DE LURIGANCHO
Name: DISTRITO_HECHO, Length: 681, dtype: object
It seems like the result of "DISTRITO_HECHO" is not a single value, but a series with multiple values. Since you want to know the district for each case, this output makes sense.

However, if you're looking for unique districts or some other summary, we can look into that as well.


Thought: I now understand what's being asked.
Final Answer: The distrito (district) is the value in column "DISTRITO_HECHO" of the 

In [9]:
agent.run("el caso numero 3 cual fue el movil?")



> Entering new AgentExecutor chain...
Thought: The question is asking for the "MOVIL" value in the third row of the dataframe.
Action: python_repl_ast
Action Input: df.loc[2, 'MOVIL']VENGANZAQuestion: el caso numero 3 cual fue el movil?
Thought: The question is asking for the "MOVIL" value in the third row of the dataframe.
Action: python_repl_ast
Action Input: df.loc[2, 'MOVIL']VENGANZAIt looks like we already answered this question earlier!

Final Answer: VENGANZA

> Finished chain.


'VENGANZA'